In [38]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
from numba import jit
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import category_encoders as ce
import numpy as np
from sklearn.linear_model import SGDRegressor
from tqdm import tqdm
import scipy
import h5py

In [43]:
train = pd.read_csv("./train_with_data_and_mean.csv")
test = pd.read_csv("./test_with_data_and_mean.csv")
sam = pd.read_csv("/root/.kaggle/competitions/ai-hack-2018-msk-gpn/sample_submission.csv")

In [42]:
train.head()

,Unnamed: 0,VID_NP,CENA_CLIENT,day,month,year,daytime_in_seconds,rounded_client_price,REGION_AZS_mean_target,VID_NP_mean_target,rounded_client_price_mean_target,month_mean_target,AZS_NUMBER_mean_target,CLIENT_mean_target
0,0,1,35.70,1,11,2016,1187,35.7,51.288309,138.503952,77.493838,82.864377,48.362749,53.476351
1,1,2,31.43,1,11,2016,5570,31.4,62.550582,32.530679,82.804494,82.864377,36.637372,60.074300
2,2,1,32.93,1,11,2016,12938,32.9,62.550582,138.503952,114.982810,82.864377,170.101413,73.436525
3,3,2,31.16,1,11,2016,4375,31.1,62.550582,32.530679,104.272030,82.864377,57.409964,60.074300
4,4,1,34.72,1,11,2016,9493,34.7,62.550582,138.503952,73.190038,82.864377,55.415038,173.086905


In [44]:
med = train.CENA_CLIENT.median()
train[train.CENA_CLIENT<10]["CENA_CLIENT"] = med
train[train.CENA_CLIENT>70]["CENA_CLIENT"] = med

test[test.CENA_CLIENT<10]["CENA_CLIENT"] = med
test[test.CENA_CLIENT>70]["CENA_CLIENT"] = med

train.drop(['year_mean_target','Unnamed: 0'], axis=1, inplace=True)
test.drop(['year_mean_target','Unnamed: 0'], axis=1, inplace=True)

/opt/intel/intelpython3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/intel/intelpython3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/intel/intelpython3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [ ]:
def bin_def(df):
    m_val = len(bin(max(df))[2:])
    arr = np.zeros((df.shape[0],m_val))
    for i in tqdm(range(len(df))):
        #print(arr[i],list(bin(df[i])[2:]))
        binar = list(bin(df[i])[2:])
        arr[i][-len(binar):] = binar
    return arr

In [ ]:
%%time
train_client_bin = bin_def(train.CLIENT.values)
test_client_bin = bin_def(test.CLIENT.values)

In [ ]:
train_region_bin = bin_def(train.REGION_AZS.values)
test_region_bin = bin_def(test.REGION_AZS.values)

In [ ]:
train_azs_bin = bin_def(train.AZS_NUMBER.values)
test_azs_bin = bin_def(test.AZS_NUMBER.values)

In [38]:
train_bin = np.hstack((train_client_bin,train_region_bin,train_azs_bin))
test_bin = np.hstack((test_client_bin,test_region_bin,test_azs_bin))

In [44]:
np.save("trainbin",train_bin)
np.save("testbin",test_bin)

In [45]:
train["AZS_REG"] = train["AZS_NUMBER"].astype("str")+train["REGION_AZS"].astype("str")
test["AZS_REG"] = test["AZS_NUMBER"].astype("str")+test["REGION_AZS"].astype("str")

In [46]:
train.head()

,CLIENT,DATA_TRANS,AZS_NUMBER,REGION_AZS,VID_NP,COL_LITR,CENA_CLIENT,day,month,year,daytime_in_seconds,rounded_client_price,REGION_AZS_mean_target,VID_NP_mean_target,rounded_client_price_mean_target,month_mean_target,AZS_NUMBER_mean_target,CLIENT_mean_target,AZS_REG
0,46219,01.11.2016 0:19:47,2335,3,1,15.00,35.70,1,11,2016,1187,35.7,51.288309,138.503952,77.493838,82.864377,48.362749,53.476351,23353
1,33792,01.11.2016 1:32:50,3356,18,2,30.00,31.43,1,11,2016,5570,31.4,62.550582,32.530679,82.804494,82.864377,36.637372,60.074300,335618
2,16215,01.11.2016 3:35:38,3381,18,1,220.00,32.93,1,11,2016,12938,32.9,62.550582,138.503952,114.982810,82.864377,170.101413,73.436525,338118
3,33792,01.11.2016 1:12:55,3303,18,2,15.00,31.16,1,11,2016,4375,31.1,62.550582,32.530679,104.272030,82.864377,57.409964,60.074300,330318
4,17075,01.11.2016 2:38:13,3276,18,1,133.67,34.72,1,11,2016,9493,34.7,62.550582,138.503952,73.190038,82.864377,55.415038,173.086905,327618


In [47]:
cat_feature = ["AZS_REG","CLIENT","AZS_NUMBER","REGION_AZS"]

In [48]:
OH = preprocessing.OneHotEncoder()

In [49]:
OH.fit(np.vstack((np.array(train.loc[:,cat_feature]),np.array(test.loc[:,cat_feature]))))
oh_tr = OH.transform(train[cat_feature])
oh_te = OH.transform(test[cat_feature])

In [50]:
y = train.COL_LITR
train.drop(["AZS_REG","CLIENT","AZS_NUMBER","REGION_AZS","DATA_TRANS","COL_LITR"],axis=1,inplace=True)
test.drop(["AZS_REG","CLIENT","AZS_NUMBER","REGION_AZS","DATA_TRANS"],axis=1,inplace=True)
train.head()

,VID_NP,CENA_CLIENT,day,month,year,daytime_in_seconds,rounded_client_price,REGION_AZS_mean_target,VID_NP_mean_target,rounded_client_price_mean_target,month_mean_target,AZS_NUMBER_mean_target,CLIENT_mean_target
0,1,35.70,1,11,2016,1187,35.7,51.288309,138.503952,77.493838,82.864377,48.362749,53.476351
1,2,31.43,1,11,2016,5570,31.4,62.550582,32.530679,82.804494,82.864377,36.637372,60.074300
2,1,32.93,1,11,2016,12938,32.9,62.550582,138.503952,114.982810,82.864377,170.101413,73.436525
3,2,31.16,1,11,2016,4375,31.1,62.550582,32.530679,104.272030,82.864377,57.409964,60.074300
4,1,34.72,1,11,2016,9493,34.7,62.550582,138.503952,73.190038,82.864377,55.415038,173.086905


In [ ]:
train = scipy.sparse.hstack((scipy.sparse.csc_matrix(train),oh_tr))
test = scipy.sparse.hstack((scipy.sparse.csc_matrix(test),oh_te))

In [ ]:
train

In [13]:
test = scipy.sparse.hstack((scipy.sparse.coo_matrix(np.array(test)),oh_te))

In [18]:
scipy.sparse.save_npz("test_sparse_2",test)

In [19]:
np.save("y",y)

In [122]:
lgb_train = lgb.Dataset(train, y)
lgb_eval = lgb.Dataset(train[30_000_000:], y[30_000_000:], reference=lgb_train)

In [17]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression_l1',
    'metric': {'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

In [154]:
def lgbm(args):
    print(args)
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression_l1',
        'metric': {'l1'},
        'num_leaves': 15 + args["num_leaves"],
        'learning_rate': 0.05,
        'feature_fraction': args["feature_fraction"],
        'bagging_fraction': args["bagging_fraction"],
        'bagging_freq': 5,
        "verbose":-1,
        "lambda_l1":args["lambda_l1"],
        "max_bin": 200 + args["max_bin"]
    }
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=40)
    return gbm

In [118]:
space = {"num_leaves":hp.randint('num_leaves', 150),
         'feature_fraction':hp.uniform('feature_fraction', 0.5, 0.99),
         'bagging_fraction':hp.uniform('bagging_fraction', 0.5, 0.99),
         "max_bin":hp.randint("max_bin",200),
         "lambda_l1":hp.uniform('lambda_l1', 0, 10)}

In [119]:
%%time
from hyperopt import fmin, tpe, hp, STATUS_OK
best = fmin(lgbm,
    space=space,
    algo=tpe.suggest, max_evals=100,verbose = 1)

{'bagging_fraction': 0.6138907563796929, 'feature_fraction': 0.6621956927156094, 'lambda_l1': 3.7815153503089824, 'max_bin': 140, 'num_leaves': 102}
[1]	valid_0's l1: 55.6486
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.872
[3]	valid_0's l1: 50.447
[4]	valid_0's l1: 48.935
[5]	valid_0's l1: 47.812
[6]	valid_0's l1: 46.8638
[7]	valid_0's l1: 46.1098
[8]	valid_0's l1: 44.8581
[9]	valid_0's l1: 43.7455
[10]	valid_0's l1: 43.5209
[11]	valid_0's l1: 42.6154
[12]	valid_0's l1: 41.8482
[13]	valid_0's l1: 41.258
[14]	valid_0's l1: 40.7085
[15]	valid_0's l1: 40.3368
[16]	valid_0's l1: 39.9984
[17]	valid_0's l1: 39.6319
[18]	valid_0's l1: 39.3908
[19]	valid_0's l1: 39.1445
[20]	valid_0's l1: 39.1172
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 39.1172
39.117155803
{'bagging_fraction': 0.662112538525932, 'feature_fraction': 0.6989803565097034, 'lambda_l1': 0.9020928181701926, 'max_bin': 138, 'num_leaves': 136}
[1]	valid_0's l1: 55.6348
Tr

[1]	valid_0's l1: 55.2637
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.3865
[3]	valid_0's l1: 50.0685
[4]	valid_0's l1: 47.9837
[5]	valid_0's l1: 46.3123
[6]	valid_0's l1: 45.3347
[7]	valid_0's l1: 44.5843
[8]	valid_0's l1: 43.4327
[9]	valid_0's l1: 42.4812
[10]	valid_0's l1: 41.7231
[11]	valid_0's l1: 41.0572
[12]	valid_0's l1: 40.4962
[13]	valid_0's l1: 40.0385
[14]	valid_0's l1: 39.6759
[15]	valid_0's l1: 39.3514
[16]	valid_0's l1: 39.0761
[17]	valid_0's l1: 38.837
[18]	valid_0's l1: 38.6949
[19]	valid_0's l1: 38.5277
[20]	valid_0's l1: 38.3963
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 38.3963
38.3963159272
{'bagging_fraction': 0.7484681793071932, 'feature_fraction': 0.6970252073161392, 'lambda_l1': 7.692027355317751, 'max_bin': 176, 'num_leaves': 95}
[1]	valid_0's l1: 55.6453
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.825
[3]	valid_0's l1: 50.4333
[4]	valid_0's l1: 48.9291
[5]	valid

[1]	valid_0's l1: 55.2657
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.3901
[3]	valid_0's l1: 50.0529
[4]	valid_0's l1: 47.9911
[5]	valid_0's l1: 46.3513
[6]	valid_0's l1: 45.3975
[7]	valid_0's l1: 44.6304
[8]	valid_0's l1: 43.5342
[9]	valid_0's l1: 42.5641
[10]	valid_0's l1: 41.788
[11]	valid_0's l1: 41.0937
[12]	valid_0's l1: 40.5446
[13]	valid_0's l1: 40.0656
[14]	valid_0's l1: 39.6544
[15]	valid_0's l1: 39.3277
[16]	valid_0's l1: 39.0355
[17]	valid_0's l1: 38.8388
[18]	valid_0's l1: 38.6666
[19]	valid_0's l1: 38.4963
[20]	valid_0's l1: 38.3724
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 38.3724
38.3723563812
{'bagging_fraction': 0.832387271172176, 'feature_fraction': 0.8300856555723979, 'lambda_l1': 8.406025275389993, 'max_bin': 161, 'num_leaves': 94}
[1]	valid_0's l1: 55.6846
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.8268
[3]	valid_0's l1: 50.3931
[4]	valid_0's l1: 48.4484
[5]	valid

[1]	valid_0's l1: 55.2647
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.3639
[3]	valid_0's l1: 50.123
[4]	valid_0's l1: 48.1529
[5]	valid_0's l1: 46.4598
[6]	valid_0's l1: 45.423
[7]	valid_0's l1: 44.6214
[8]	valid_0's l1: 43.5222
[9]	valid_0's l1: 42.5377
[10]	valid_0's l1: 41.7634
[11]	valid_0's l1: 41.0857
[12]	valid_0's l1: 40.5575
[13]	valid_0's l1: 40.0924
[14]	valid_0's l1: 39.6983
[15]	valid_0's l1: 39.3834
[16]	valid_0's l1: 39.105
[17]	valid_0's l1: 38.887
[18]	valid_0's l1: 38.7335
[19]	valid_0's l1: 38.5631
[20]	valid_0's l1: 38.4579
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 38.4579
38.4579052575
{'bagging_fraction': 0.6695817747540975, 'feature_fraction': 0.9874443468994732, 'lambda_l1': 3.3865058286685756, 'max_bin': 117, 'num_leaves': 63}
[1]	valid_0's l1: 55.2807
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.4296
[3]	valid_0's l1: 50.1259
[4]	valid_0's l1: 48.165
[5]	valid_0

[1]	valid_0's l1: 55.3057
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.4677
[3]	valid_0's l1: 50.1419
[4]	valid_0's l1: 48.2312
[5]	valid_0's l1: 46.6123
[6]	valid_0's l1: 45.263
[7]	valid_0's l1: 44.0751
[8]	valid_0's l1: 43.1083
[9]	valid_0's l1: 42.2324
[10]	valid_0's l1: 41.585
[11]	valid_0's l1: 40.9799
[12]	valid_0's l1: 40.4806
[13]	valid_0's l1: 40.0526
[14]	valid_0's l1: 39.7096
[15]	valid_0's l1: 39.4047
[16]	valid_0's l1: 39.1631
[17]	valid_0's l1: 38.9417
[18]	valid_0's l1: 38.7706
[19]	valid_0's l1: 38.6323
[20]	valid_0's l1: 38.5059
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 38.5059
38.5059241148
{'bagging_fraction': 0.5480171459407621, 'feature_fraction': 0.5016939237039316, 'lambda_l1': 1.4747488671638953, 'max_bin': 130, 'num_leaves': 90}
[1]	valid_0's l1: 55.6506
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.8983
[3]	valid_0's l1: 50.4897
[4]	valid_0's l1: 48.8039
[5]	vali

[1]	valid_0's l1: 55.2626
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.3825
[3]	valid_0's l1: 50.0673
[4]	valid_0's l1: 47.9966
[5]	valid_0's l1: 46.3133
[6]	valid_0's l1: 44.9127
[7]	valid_0's l1: 43.6855
[8]	valid_0's l1: 42.677
[9]	valid_0's l1: 41.8139
[10]	valid_0's l1: 41.1702
[11]	valid_0's l1: 40.5774
[12]	valid_0's l1: 40.0596
[13]	valid_0's l1: 39.6703
[14]	valid_0's l1: 39.311
[15]	valid_0's l1: 39.0206
[16]	valid_0's l1: 38.8078
[17]	valid_0's l1: 38.5884
[18]	valid_0's l1: 38.4193
[19]	valid_0's l1: 38.2862
[20]	valid_0's l1: 38.1712
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 38.1712
38.1711908486
{'bagging_fraction': 0.8795880967509004, 'feature_fraction': 0.6430962099832638, 'lambda_l1': 5.782240376588786, 'max_bin': 20, 'num_leaves': 120}
[1]	valid_0's l1: 55.6369
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.8716
[3]	valid_0's l1: 50.4882
[4]	valid_0's l1: 48.9236
[5]	valid

[1]	valid_0's l1: 55.6792
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.843
[3]	valid_0's l1: 50.4453
[4]	valid_0's l1: 48.9414
[5]	valid_0's l1: 47.864
[6]	valid_0's l1: 46.9941
[7]	valid_0's l1: 46.2264
[8]	valid_0's l1: 44.9468
[9]	valid_0's l1: 43.7808
[10]	valid_0's l1: 42.8267
[11]	valid_0's l1: 41.9989
[12]	valid_0's l1: 41.305
[13]	valid_0's l1: 40.7441
[14]	valid_0's l1: 40.2868
[15]	valid_0's l1: 39.8622
[16]	valid_0's l1: 39.5213
[17]	valid_0's l1: 39.2395
[18]	valid_0's l1: 39.0382
[19]	valid_0's l1: 38.8399
[20]	valid_0's l1: 38.8035
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 38.8035
38.8034830884
{'bagging_fraction': 0.8008587063226487, 'feature_fraction': 0.9263903107594698, 'lambda_l1': 1.2526034500710237, 'max_bin': 92, 'num_leaves': 114}
[1]	valid_0's l1: 55.2642
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.3854
[3]	valid_0's l1: 50.0684
[4]	valid_0's l1: 47.9823
[5]	valid

[1]	valid_0's l1: 55.2661
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.3862
[3]	valid_0's l1: 50.0592
[4]	valid_0's l1: 47.9799
[5]	valid_0's l1: 46.3153
[6]	valid_0's l1: 45.387
[7]	valid_0's l1: 44.6219
[8]	valid_0's l1: 43.4733
[9]	valid_0's l1: 42.5109
[10]	valid_0's l1: 41.7586
[11]	valid_0's l1: 41.107
[12]	valid_0's l1: 40.5371
[13]	valid_0's l1: 40.0641
[14]	valid_0's l1: 39.6611
[15]	valid_0's l1: 39.3449
[16]	valid_0's l1: 39.0619
[17]	valid_0's l1: 38.8255
[18]	valid_0's l1: 38.6614
[19]	valid_0's l1: 38.4733
[20]	valid_0's l1: 38.3583
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 38.3583
38.3582784324
{'bagging_fraction': 0.6220182105758103, 'feature_fraction': 0.8863638738016147, 'lambda_l1': 9.934444276375402, 'max_bin': 163, 'num_leaves': 31}
[1]	valid_0's l1: 55.3072
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.4684
[3]	valid_0's l1: 50.1483
[4]	valid_0's l1: 48.2559
[5]	valid

[1]	valid_0's l1: 55.2563
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.4042
[3]	valid_0's l1: 50.0456
[4]	valid_0's l1: 47.9633
[5]	valid_0's l1: 46.2658
[6]	valid_0's l1: 45.3364
[7]	valid_0's l1: 44.5836
[8]	valid_0's l1: 43.4335
[9]	valid_0's l1: 42.465
[10]	valid_0's l1: 41.691
[11]	valid_0's l1: 40.9995
[12]	valid_0's l1: 40.458
[13]	valid_0's l1: 39.9715
[14]	valid_0's l1: 39.5712
[15]	valid_0's l1: 39.2585
[16]	valid_0's l1: 38.9727
[17]	valid_0's l1: 38.7611
[18]	valid_0's l1: 38.5885
[19]	valid_0's l1: 38.4182
[20]	valid_0's l1: 38.291
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 38.291
38.2910050651
{'bagging_fraction': 0.7908277119660644, 'feature_fraction': 0.9587966772001055, 'lambda_l1': 2.8321545772862384, 'max_bin': 23, 'num_leaves': 11}
[1]	valid_0's l1: 55.4464
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.9046
[3]	valid_0's l1: 50.6068
[4]	valid_0's l1: 48.648
[5]	valid_0's

[1]	valid_0's l1: 55.6853
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.9728
[3]	valid_0's l1: 50.5726
[4]	valid_0's l1: 49.0613
[5]	valid_0's l1: 47.9109
[6]	valid_0's l1: 46.9758
[7]	valid_0's l1: 46.2199
[8]	valid_0's l1: 44.924
[9]	valid_0's l1: 43.7989
[10]	valid_0's l1: 42.8464
[11]	valid_0's l1: 42.058
[12]	valid_0's l1: 41.393
[13]	valid_0's l1: 40.864
[14]	valid_0's l1: 40.4037
[15]	valid_0's l1: 40.0094
[16]	valid_0's l1: 39.6992
[17]	valid_0's l1: 39.423
[18]	valid_0's l1: 39.2339
[19]	valid_0's l1: 39.0192
[20]	valid_0's l1: 38.9917
Did not meet early stopping. Best iteration is:
[20]	valid_0's l1: 38.9917
38.9917201295
{'bagging_fraction': 0.7928831618383976, 'feature_fraction': 0.7741112812743088, 'lambda_l1': 5.740310157010619, 'max_bin': 106, 'num_leaves': 87}
[1]	valid_0's l1: 55.6437
Training until validation scores don't improve for 2 rounds.
[2]	valid_0's l1: 52.9114
[3]	valid_0's l1: 50.4822
[4]	valid_0's l1: 48.9608
[5]	valid_0'

In [121]:
best

{'bagging_fraction': 0.7330235494960213,
 'feature_fraction': 0.9588967250415795,
 'lambda_l1': 4.469728994283145,
 'max_bin': 79,
 'num_leaves': 116}

In [5]:
%%time
sam = pd.read_csv("/root/.kaggle/competitions/ai-hack-2018-msk-gpn/sample_submission.csv")
train = scipy.sparse.load_npz("./train_sparse.npz")
test = scipy.sparse.load_npz("./test_sparse.npy")
y = np.load("./y.npy")

TypeError: load_npz() got an unexpected keyword argument 'allow_pickle'

In [14]:
rf = RandomForestRegressor(oob_score=True, n_jobs=5, n_estimators=10,random_state=42)

In [ ]:
be

In [16]:
rf.oob_score_

AttributeError: 'RandomForestRegressor' object has no attribute 'oob_score_'

In [155]:
%%time 
ev = lgbm(best)

{'bagging_fraction': 0.7330235494960213, 'feature_fraction': 0.9588967250415795, 'lambda_l1': 4.469728994283145, 'max_bin': 79, 'num_leaves': 116}
CPU times: user 1h 27min 43s, sys: 3min 41s, total: 1h 31min 24s
Wall time: 3min 8s


In [156]:
%%time
pred = ev.predict(test)

CPU times: user 9.51 s, sys: 232 ms, total: 9.74 s
Wall time: 329 ms


In [157]:
sam["COL_LITR"] = pred

In [158]:
sam.to_csv("lgb_40.csv",index=False)

In [145]:
print("ok")

ok


In [159]:
! /opt/intel/intelpython3/bin/kaggle competitions submit -c ai-hack-2018-msk-gpn -f lgb_40.csv -m "OK?"

Successfully submitted to Хакатон AI.Hack Москва. 